In [ ]:
# run init.ipynb before

In [2]:
bucket = 'mstoffel'
subfolder = 'mstoffel/t15264971/measurements_c8y_Acceleration/'

In [3]:
import numpy as np
import pandas as pd
import os
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
import tensorflow as tf
import tf2onnx
import onnx

In [4]:
##
# the following path is hardcoded
# TODO find a more clever way to identify the data
##
S3_BCUKET = "mstoffel/mstoffel/t15264971"
DATA_PATH = f"{S3_BCUKET}/measurements_c8y_Acceleration/chunk_P1_108_109_1686268800000_1687860401522_295601521/0_2023/0_06/0_27/0_0_0.parquet"
MODEL_NAME = 'acceleration_model.onnx'
FULL_MODEL_PATH = f"{S3_BCUKET}/model/{MODEL_NAME}"
fs = s3fs.S3FileSystem()

pf = pq.ParquetDataset(
    DATA_PATH,
    filesystem=fs)

In [5]:
##
# something along the line ...
##
#s3_filepath = 'mstoffel/mstoffel/t15264971/measurements_c8y_Acceleration/chunk_P1_108_109_1686268800000_1687860401522_295601521/0_2023/0_06/0_27'
#s3_filepaths = [path for path in fs.ls(s3_filepath)
#                if path.endswith('.parquet')]


# s3_filepaths

In [6]:
# show schema
pf.schema

id: string
creationTime: timestamp[ms, tz=UTC]
creationTimeOffset: int32
creationTimeWithOffset: timestamp[ms, tz=UTC]
YEAR: string
MONTH: string
DAY: string
time: timestamp[ms, tz=UTC]
timeOffset: int32
timeWithOffset: timestamp[ms, tz=UTC]
source: string
type: string
c8y_Acceleration.accelerationX.label: string
c8y_Acceleration.accelerationX.value: double
c8y_Acceleration.accelerationY.label: string
c8y_Acceleration.accelerationY.value: double
c8y_Acceleration.accelerationZ.label: string
c8y_Acceleration.accelerationZ.value: double
-- schema metadata --
is.date.correct: 'true'
dremio.arrow.schema.2.1: '{
  "fields" : [ {
    "name" : "id",
    "null' + 2759
dremio.version: '19.11.2-202301280043390504-e5820c5a'

In [7]:
# convert parquet datatset to pandas
data = pf.read().to_pandas()
data.sample(5)

,id,creationTime,creationTimeOffset,creationTimeWithOffset,YEAR,MONTH,DAY,time,timeOffset,timeWithOffset,source,type,c8y_Acceleration.accelerationX.label,c8y_Acceleration.accelerationX.value,c8y_Acceleration.accelerationY.label,c8y_Acceleration.accelerationY.value,c8y_Acceleration.accelerationZ.label,c8y_Acceleration.accelerationZ.value
551,58567575,2023-06-27 10:00:23.744000+00:00,0,2023-06-27 10:00:23.744000+00:00,2023,06,27,2023-04-03 07:37:00+00:00,120,2023-04-03 09:37:00+00:00,5558565188,c8y_Acceleration,moving,0.508636,moving,0.606796,moving,-0.708145
172,58571316,2023-06-27 10:00:30.610000+00:00,0,2023-06-27 10:00:30.610000+00:00,2023,06,27,2023-04-03 07:36:13.500000+00:00,120,2023-04-03 09:36:13.500000+00:00,5558565188,c8y_Acceleration,moving,0.761322,moving,0.104385,moving,-1.215836
428,58567011,2023-06-27 10:00:29.936000+00:00,0,2023-06-27 10:00:29.936000+00:00,2023,06,27,2023-04-03 07:36:59.500000+00:00,120,2023-04-03 09:36:59.500000+00:00,5558565188,c8y_Acceleration,moving,1.101227,moving,-0.117447,moving,-0.600128
185,58567002,2023-06-27 10:00:28.990000+00:00,0,2023-06-27 10:00:28.990000+00:00,2023,06,27,2023-04-03 07:25:06.500000+00:00,120,2023-04-03 09:25:06.500000+00:00,5558565188,c8y_Acceleration,sitting,0.010040,sitting,0.023682,sitting,0.997513
399,58567025,2023-06-27 10:00:31.363000+00:00,0,2023-06-27 10:00:31.363000+00:00,2023,06,27,2023-04-03 07:36:47.500000+00:00,120,2023-04-03 09:36:47.500000+00:00,5558565188,c8y_Acceleration,moving,-0.080902,moving,0.095108,moving,1.228516


In [8]:
#remove time column from the source
data.pop(data.columns[0])
data.sample(5)

,creationTime,creationTimeOffset,creationTimeWithOffset,YEAR,MONTH,DAY,time,timeOffset,timeWithOffset,source,type,c8y_Acceleration.accelerationX.label,c8y_Acceleration.accelerationX.value,c8y_Acceleration.accelerationY.label,c8y_Acceleration.accelerationY.value,c8y_Acceleration.accelerationZ.label,c8y_Acceleration.accelerationZ.value
119,2023-06-27 10:00:26.907000+00:00,0,2023-06-27 10:00:26.907000+00:00,2023,06,27,2023-04-03 07:37:28.500000+00:00,120,2023-04-03 09:37:28.500000+00:00,5558565188,c8y_Acceleration,moving,0.107346,moving,0.017365,moving,1.277527
6,2023-06-27 10:00:15.796000+00:00,0,2023-06-27 10:00:15.796000+00:00,2023,06,27,2023-04-03 07:24:49+00:00,120,2023-04-03 09:24:49+00:00,5558565188,c8y_Acceleration,sitting,0.010559,sitting,0.024368,sitting,0.996017
109,2023-06-27 10:00:17.283000+00:00,0,2023-06-27 10:00:17.283000+00:00,2023,06,27,2023-04-03 07:25:38+00:00,120,2023-04-03 09:25:38+00:00,5558565188,c8y_Acceleration,sitting,0.011383,sitting,0.025131,sitting,0.997314
400,2023-06-27 10:00:26.192000+00:00,0,2023-06-27 10:00:26.192000+00:00,2023,06,27,2023-04-03 07:25:16+00:00,120,2023-04-03 09:25:16+00:00,5558565188,c8y_Acceleration,sitting,0.010651,sitting,0.024490,sitting,0.996017
393,2023-06-27 10:00:33.723000+00:00,0,2023-06-27 10:00:33.723000+00:00,2023,06,27,2023-04-03 07:24:47+00:00,120,2023-04-03 09:24:47+00:00,5558565188,c8y_Acceleration,sitting,0.010208,sitting,0.023712,sitting,0.997528


In [9]:

# drop unused column and rename colum
data = data.drop( ['creationTime', 'creationTimeOffset','creationTimeWithOffset', 'YEAR', 'MONTH', 'DAY', 'time', 'timeOffset', 'timeWithOffset', 'source', 'type',  'c8y_Acceleration.accelerationX.label', 'c8y_Acceleration.accelerationY.label'  ], axis = 1)

data = data.rename(columns={"c8y_Acceleration.accelerationZ.label": "label" ,"c8y_Acceleration.accelerationX.value": "accelerationX", "c8y_Acceleration.accelerationY.value": "accelerationY","c8y_Acceleration.accelerationZ.value": "accelerationZ"})

data.sample(5)

# pop the label columns
#labels = data.pop("label")
#print(labels)

,accelerationX,accelerationY,label,accelerationZ
44,0.365356,-0.002930,moving,1.358551
431,0.007843,0.024094,sitting,0.997192
124,-0.270889,0.642654,moving,0.438583
448,0.009476,0.023788,sitting,0.998154
13,0.166626,0.421875,moving,0.610367


In [10]:
# pop the label columns
labels = data.pop("label")
print(labels)

0       moving
1       moving
2      sitting
3       moving
4       moving
        ...   
739    sitting
740    sitting
741    sitting
742    sitting
743    sitting
Name: label, Length: 744, dtype: object


In [11]:
# normalize data
data = (data - data.mean()) / data.std() 
data.sample(5)

,accelerationX,accelerationY,accelerationZ
657,2.260148,-1.410240,-3.161313
219,1.595100,-0.071237,-0.396602
337,1.644598,0.472659,0.447629
363,-0.194140,-0.502176,0.525082
44,0.827674,-0.603119,1.063866


In [12]:
# Reshape data to 3D array
data = data.values.reshape((-1, 3, 1))
print(data)

[[[-1.2386366 ]
  [ 2.13942496]
  [-1.0925436 ]]

 [[ 0.953395  ]
  [ 3.02557342]
  [-0.81866035]]

 [[-0.1999426 ]
  [-0.49619956]
  [ 0.52519592]]

 ...

 [[-0.19827218]
  [-0.49475062]
  [ 0.52530962]]

 [[-0.19739301]
  [-0.50326315]
  [ 0.52412713]]

 [[-0.1933928 ]
  [-0.49601845]
  [ 0.52580991]]]


In [13]:
# Convert labels to binary
labels = np.where(labels=="moving", 1, 0)

In [14]:
# Split data into train and test sets
split = int(len(data) * 0.8)
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]

In [15]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3, 1)),
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Reshape((3, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64)

Epoch 1/30
10/10 [==============================] - 1s 26ms/step - loss: 0.6745 - accuracy: 0.5950 - val_loss: 0.6214 - val_accuracy: 0.5302
Epoch 2/30
10/10 [==============================] - 0s 8ms/step - loss: 0.5980 - accuracy: 0.6992 - val_loss: 0.5602 - val_accuracy: 0.9866
Epoch 3/30
10/10 [==============================] - 0s 7ms/step - loss: 0.5440 - accuracy: 0.8571 - val_loss: 0.5042 - val_accuracy: 0.9664
Epoch 4/30
10/10 [==============================] - 0s 5ms/step - loss: 0.4920 - accuracy: 0.9361 - val_loss: 0.4511 - val_accuracy: 0.9396
Epoch 5/30
10/10 [==============================] - 0s 7ms/step - loss: 0.4422 - accuracy: 0.9227 - val_loss: 0.4021 - val_accuracy: 0.9060
Epoch 6/30
10/10 [==============================] - 0s 6ms/step - loss: 0.3971 - accuracy: 0.9160 - val_loss: 0.3584 - val_accuracy: 0.8993
Epoch 7/30
10/10 [==============================] - 0s 6ms/step - loss: 0.3575 - accuracy: 0.9126 - val_loss: 0.3214 - val_accuracy: 0.9060
Epoch 8/30
10/10 [=

In [18]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

5/5 [==============================] - 0s 2ms/step - loss: 0.0746 - accuracy: 0.9866
Test accuracy: 0.9865771532058716


In [19]:
# Use the model to predict label for a single row of data
single_row = np.array([[[0.070770263671875,0.7673492431640625,0.751739501953125]]]).reshape((1, 3, 1))
#single_row = np.array([1.0,1.0,1.0]).reshape((1, 3, 1))
prediction = model.predict(single_row)
prediction 

1/1 [==============================] - 0s 95ms/step


array([[0.9979492]], dtype=float32)

In [20]:
#MODEL_DIR = tempfile.gettempdir()
version = 1
#export_path = os.path.join(MODEL_DIR, str(version))
#print('export_path = {}\n'.format(export_path))

export_path = '../model/cumulocity_classification_model'

# Export the model as a SavedModel
tf.saved_model.save(model, export_path)


print('\nSaved model:' + export_path)
#!ls -l {export_path}

INFO:tensorflow:Assets written to: ../model/cumulocity_classification_model/assets


INFO:tensorflow:Assets written to: ../model/cumulocity_classification_model/assets



Saved model:../model/cumulocity_classification_model


In [21]:
#import glob
#print(glob.glob("../model/cumulocity_classification_model_3/0000001/*"))

In [22]:
local_model_path = f"{export_path}/model.onnx"
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, local_model_path)

In [23]:
#import glob
#print(glob.glob("../model/cumulocity_classification_model_3/*"))

In [29]:

file_name = 'model/acceleration_model.onnx'
S3_MODEL_PATH = f"{S3_BCUKET}/{file_name}"''

s3 = s3fs.S3FileSystem(
    anon=False,
    use_ssl=True,
    client_kwargs={
        "region_name": 'eu-central-1',
        "verify": True,
    }
)
s3.put(local_model_path, S3_MODEL_PATH)



In [30]:
S3_MODEL_PATH

'mstoffel/mstoffel/t15264971/model/acceleration_model.onnx'

In [31]:
local_model_path

'../model/cumulocity_classification_model/model.onnx'